In [1]:
import os
%pwd

'c:\\Users\\lenovo\\Desktop\\Study\\AI_Projects\\TextSummarizer\\research'

In [2]:
os.chdir("../")

In [3]:
%pwd

'c:\\Users\\lenovo\\Desktop\\Study\\AI_Projects\\TextSummarizer'

Config Entity Update:

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainerConfig:
    root_dir: Path
    data_path: Path
    model_ckpt: Path
    num_train_epochs: int
    warmup_steps: int
    per_device_train_batch_size: int
    weight_decay: float
    logging_steps: int
    evaluation_strategy: int
    eval_steps: int
    save_steps: float
    gradient_accumulation_steps: int

Configuration Manager Update:

In [5]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager():
        def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH):

            self.config = read_yaml(config_filepath)
            self.params = read_yaml(params_filepath)

            create_directories([self.config.artifacts_root])

        def get_model_trainer_config(self)-> ModelTrainerConfig:
            config = self.config.model_trainer
            params = self.params.TrainingArguments

            create_directories([config.root_dir])

            model_trainer_config=ModelTrainerConfig(
                root_dir=config.root_dir,
                data_path=config.data_path,
                model_ckpt = config.model_ckpt,
                num_train_epochs = params.num_train_epochs,
                warmup_steps = params.warmup_steps,
                per_device_train_batch_size = params.per_device_train_batch_size,
                weight_decay = params.weight_decay,
                logging_steps = params.logging_steps,
                evaluation_strategy = params.evaluation_strategy,
                eval_steps = params.evaluation_strategy,
                save_steps = params.save_steps,
                gradient_accumulation_steps = params.gradient_accumulation_steps
            )
            return model_trainer_config

Component Update:

In [7]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
from transformers import TrainingArguments, Trainer
import torch
from datasets import load_from_disk



c:\Users\lenovo\Desktop\Study\AI_Projects\TextSummarizer\tsvenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-04 14:26:57,283: INFO: config: PyTorch version 2.6.0 available.]


In [8]:
class ModelTrainer():
    def __init__(self, config:ModelTrainerConfig):
        self.config = config

    def train(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.model_ckpt)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_ckpt).to(device)
        seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_pegasus)

        #loading the dataset
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        training_args = TrainingArguments(output_dir=self.config.root_dir,
                                          num_train_epochs= self.config.num_train_epochs,
                                          warmup_steps=self.config.warmup_steps,
                                          per_device_train_batch_size=self.config.per_device_train_batch_size,
                                          weight_decay=self.config.weight_decay,
                                          logging_steps=self.config.logging_steps,
                                          eval_strategy=self.config.evaluation_strategy,
                                          eval_steps=self.config.eval_steps,
                                          save_steps=self.config.save_steps,
                                          gradient_accumulation_steps=self.config.gradient_accumulation_steps)

        trainer = Trainer(model=model_pegasus,TrainingArguments=training_args,
                          data_collator=seq2seq_data_collator,
                          train_dataset=dataset_samsum_pt["test"],
                          eval_dataset=dataset_samsum_pt["validation"])
        
        trainer.train()

        ## Save model
        model_pegasus.save_pretrained(os.path.join(self.config.root_dir,"pegasus-samsum-model"))
        ## Save tokenizer
        tokenizer.save_pretrained(os.path.join(self.config.root_dir,"tokenizer"))



In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.50.3
Uninstalling transformers-4.50.3:
  Successfully uninstalled transformers-4.50.3
Found existing installation: accelerate 1.6.0
Uninstalling accelerate-1.6.0:
  Successfully uninstalled accelerate-1.6.0
  Using cached transformers-4.50.3-py3-none-any.whl.metadata (39 kB)
  Using cached accelerate-1.6.0-py3-none-any.whl.metadata (19 kB)
Using cached transformers-4.50.3-py3-none-any.whl (10.2 MB)
Using cached accelerate-1.6.0-py3-none-any.whl (354 kB)


: 

Model trainer Pipeline Update:

In [ ]:
config = ConfigurationManager()
model_trainer_config = config.get_model_trainer_config()
model_trainer = ModelTrainer(config=model_trainer_config)
model_trainer.train()

[2025-04-04 14:28:32,555: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-04-04 14:28:32,612: INFO: common: yaml file: params.yaml loaded successfully]
[2025-04-04 14:28:32,621: INFO: common: created directory at: artifacts]
[2025-04-04 14:28:32,624: INFO: common: created directory at: artifacts/model_trainer]
